In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# A small DIY transformer

by Adam Shai
2023.01.21

following Karpathy's youtube video:  https://www.youtube.com/watch?v=kCc8FmEb1nY
shakespear text data is called `shakespear_data.txt`

## The Shakespear Data
this is the "tiny shakeseapr" dataset

In [2]:
# open the shakespear_data.txt file and read in
with open("shakespear_data.txt") as f:
    data = f.read()

# print the length of the data
print("the length of the data is {} characters".format(len(data)))

the length of the data is 1115393 characters


In [3]:
# print the first 250 characters in the data
print(data[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
# make the data a list of characters
chars = list(set(data))
vocab_size = len(chars)

# print the length of the vocabulary
print("the length of the vocabulary is {} characters".format(len(chars)))



the length of the vocabulary is 65 characters


In [6]:
# make a dictionary to convert from characters to integers
char_to_int = {ch:i for i,ch in enumerate(chars)}

# make a dictionary to convert from integers to characters
int_to_char = {i:ch for i,ch in enumerate(chars)}

# convert the characters to integers
int_data = [char_to_int[ch] for ch in data]

# test this out by taking a random slice of the data, converting to ints, then back to chars
# print out each step
rand_slice = np.random.randint(0, len(data)-20)
print("the random slice is: ", data[rand_slice:rand_slice+20])
print("the random slice as ints is: ", int_data[rand_slice:rand_slice+20])
print("the random slice as chars is: ", ''.join([int_to_char[i] for i in int_data[rand_slice:rand_slice+20]]))

the random slice is:  usin, farewell; and,
the random slice as ints is:  [2, 26, 18, 51, 43, 32, 7, 21, 41, 39, 52, 39, 9, 9, 23, 32, 21, 51, 17, 43]
the random slice as chars is:  usin, farewell; and,


In [64]:
torch.manual_seed(123)

context_length = 8
batch_size = 4

# make a function to generate a batch of data
def generate_batch(data, context_length, batch_size):
    # we want to return  2 tensors, x and y
    # x will be the input to the network, y will be the target
    # x will be a tensor of size (batch_size, context_length)
    # y will be a tensor of size (batch_size, context_length)
    # y will be the same as x, but shifted over by one character

    x = torch.zeros((batch_size, context_length), dtype=torch.long)
    y = torch.zeros((batch_size, context_length), dtype=torch.long)

    # get a random starting point for each batch, use torch
    rand_starts = torch.randint(0, len(data)-context_length, (batch_size,))

    # fill in the x and y tensors
    for i, start in enumerate(rand_starts):
        x[i,:] = torch.tensor(data[start:start+context_length])
        y[i,:] = torch.tensor(data[start+1:start+context_length+1])

    return x, y


In [65]:

# test the function
x, y = generate_batch(int_data, context_length, batch_size)

# the batch is
print("the first batch is\n ", x)
print("with targets\n ", y)

# convert to characters, but be careful about torch tensors and numpy arrays
print("the first batch as characters is\t ", ''.join([int_to_char[i] for i in x[0,:].numpy()]))
print("the first target as characters is\t ", ''.join([int_to_char[i] for i in y[0,:].numpy()]))


the first batch is
  tensor([[37,  5, 31,  2, 32, 26,  5, 21],
        [10, 32,  5, 39, 51, 36, 39, 32],
        [32,  6, 39,  9, 18, 39, 24, 39],
        [32, 21, 32,  6,  9, 31, 52, 32]])
with targets
  tensor([[ 5, 31,  2, 32, 26,  5, 21,  9],
        [32,  5, 39, 51, 36, 39, 32, 21],
        [ 6, 39,  9, 18, 39, 24, 39, 32],
        [21, 32,  6,  9, 31, 52, 32, 37]])
the first batch as characters is	  thou sha
the first target as characters is	  hou shal


In [112]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super().__init__()

        # define the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, idx_inputs, idx_targets=None):
        # idx_inputs is a tensor of size (batch_size, context_length)
        # where the values are the indices of the characters
        # the output will be a tensor of size (batch_size, context_length, embedding_dim)
        # idx_targets is a tensor of size (batch_size, context_length)
        # get the embeddings for the inputs
        logits = self.embedding(idx_inputs) # (batch_size, context_length, embedding_dim)
        # take the cross entropy. we need to reshape for torch
        # the logits need to be (batch_size, vocab_size, context_length)
        # the targets need to be (batch_size, context_length)
        if idx_targets is None:
            return logits
        else:
            # to compute the loss we need the logits to be (batch_size*context_length, vocab_size)
            # and we need targets to be (batch_size*context_length)
            loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), idx_targets.view(-1))
            return logits, loss

    def predict(self, idx_inputs, max_steps=100):
        # idx_inputs is a tensor of size (batch_size, context_length)
        # where the values are the indices of the characters
        # the output will be a tensor of size (batch_size, context_length, embedding_dim)
        for i in range(max_steps):
            # get the embeddings for the inputs
            logits = self(idx_inputs) # (batch_size, context_length, embedding_dim)
            # get the logits for the last character, which is the prediction
            logits = logits[:,-1,:] # (batch_size, embedding_dim)
            # get the prediction by taking softmax
            probs = F.softmax(logits, dim=1) # (batch_size, vocab_size)
            # get the prediction by sampling from the distribution
            next_char = torch.multinomial(probs, num_samples=1) # (batch_size, 1)
            # add the prediction to the input
            idx_inputs = torch.cat((idx_inputs, next_char), dim=1) # (batch_size, context_length+1)

        return idx_inputs



    
    

In [114]:
# test the model
model = BigramLanguageModel(vocab_size, vocab_size)
idx_inputs, idx_targets = generate_batch(int_data, context_length, batch_size)
logits, loss = model(idx_inputs, idx_targets)
print("the loss is ", loss.item())


the loss is  4.749288558959961


In [115]:

# test the predict function
idx_inputs, idx_targets = generate_batch(int_data, context_length, batch_size)
idx_outputs = model.predict(idx_inputs)
# convert to characters, but be careful about torch tensors and numpy arrays
print("the output as characters is\n ", ''.join([int_to_char[i] for i in idx_outputs[3,:].numpy()]))


the output as characters is
   CitizenMqiudGGoEPYrCbGRG't$KZ,MBkp.I'M?
Z?A!cqSvj:O?JAG3N&qAQi;mr.c MBeG,Fe!zl,xc-!j
b:YTdPOPEZ!nDSB 
ZN?.j


In [117]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# train the model
num_epochs = 10000
batch_size = 32
for epoch in range(num_epochs):
    # get the data
    idx_inputs, idx_targets = generate_batch(int_data, context_length, batch_size)
    # zero the gradients
    optimizer.zero_grad(set_to_none=True)
    # get the logits
    logits, loss = model(idx_inputs, idx_targets)
    # take the gradient step
    loss.backward()
    optimizer.step()
    # print the loss
print("the loss is ", loss.item())

the loss is  2.5120346546173096


In [119]:
# now make a prediction
idx_inputs, idx_targets = generate_batch(int_data, context_length, batch_size)
idx_outputs = model.predict(idx_inputs,1000)
# convert to characters, but be careful about torch tensors and numpy arrays
print("the output as characters is\n ", ''.join([int_to_char[i] for i in idx_outputs[3,:].numpy()]))

the output as characters is
   others'samathan:
T:
Whighatowes tthewXWamy tiss g d's IOLISh tengo t, ce sthmurspair su Charisithematheno hillly!boorm hat t anowhes t g hen mave y, t wLowhashal wio:
T:
Than pes s,
ErppKI thee CO,
ielio aikiveve VAs
OMerin cHemimether yshe, bannincrervel h GRKRI:
RWhe: te n sthyoad,
Whigro OR:
ARINo da me to acernou thaspe-sstr TZHabad am
MI ghee y be m poutotr, thons goss d mastonsht me thixf pode ke treoou, sssply ouingu fe s, at:
Wat m atheang IUnousouthin th.
MEd gh rd veathenof te th atowe?
stit CHETy e fe whiof horyr qut? w m raversitA:
Anist, wichy, wil thigmuthrr yo heo-bebes?
Thashire
OS heeatin ay isood inoded wesin we canst, tybou, t s ltl ch teme have ghthaiowh t e t renpoo ' stivet f blowinthe thastour banu havas be?
CAs t  I ce,
O:


UKESTher acoy in ch Str? puppas a m dst t s we:
A:
ll thi, su sireongotimmeeramblvZEdirveraru y imallyou t y
Wiotece;
GLO:MININou torouitere s t Mag ybe
R:
I ONofesther t
My g-menken g-m ord s th cad,e t d m, d

In [120]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('shakespear_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4112, val loss 4.4015
step 100: train loss 2.6576, val loss 2.6632
step 200: train loss 2.5119, val loss 2.5024
step 300: train loss 2.4154, val loss 2.4308
step 400: train loss 2.3515, val loss 2.3670
step 500: train loss 2.3018, val loss 2.3231
step 600: train loss 2.2551, val loss 2.2614
step 700: train loss 2.2143, val loss 2.2239
step 800: train loss 2.1595, val loss 2.1904
step 900: train loss 2.1410, val loss 2.1511
step 1000: train loss 2.1018, val loss 2.1311
step 1100: train loss 2.0636, val loss 2.1141
step 1200: train loss 2.0482, val loss 2.0955
step 1300: train loss 2.0200, val loss 2.0648
step 1400: train loss 2.0009, val loss 2.0459
step 1500: train loss 1.9857, val loss 2.0371
step 1600: train loss 1.9702, val loss 2.0429
step 1700: train loss 1.9513, val loss 2.0298
step 1800: train loss 1.9277, val loss 2.0230
step 1900: train loss 1.9139, val loss 1.9868
step 2000: train loss 1.9046, val loss 1.9923
step 2100: train loss 1.

In [1]:
131*131

17161

In [2]:
113*113

12769